Import the libraries:

In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

 Define image properties

In [2]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

Prepare dataset for training model

In [3]:
filenames=os.listdir(r"C:\Users\abhay\OneDrive\Desktop\train\train")

categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)

df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

Create the neural net model

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization

model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

Analyzing model

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 126, 126, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 61, 61, 64)        2

Define callbacks and learning rate

In [6]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

Manage data

In [7]:
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15

Training and validation data generator

In [8]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 r"C:\Users\abhay\OneDrive\Desktop\train\train",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    r"C:\Users\abhay\OneDrive\Desktop\train\train", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)

test_generator = train_datagen.flow_from_dataframe(train_df,
                                                 r"C:\Users\abhay\OneDrive\Desktop\test1\test1",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Found 0 validated image filenames belonging to 0 classes.


C:\Users\abhay\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\preprocessing\image.py:1137: UserWarning: Found 20000 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [9]:
epochs=10
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

C:\Users\abhay\AppData\Local\Temp\ipykernel_28516\2528005629.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10


1333/1333 [==============================] - 1362s 1s/step - loss: 0.7457 - accuracy: 0.6277 - val_loss: 0.5603 - val_accuracy: 0.7067 - lr: 0.0010
Epoch 2/10
1333/1333 [==============================] - 750s 563ms/step - loss: 0.5580 - accuracy: 0.7217 - val_loss: 0.7448 - val_accuracy: 0.6533 - lr: 0.0010
Epoch 3/10
1333/1333 [==============================] - 627s 470ms/step - loss: 0.5045 - accuracy: 0.7590 - val_loss: 0.6427 - val_accuracy: 0.7465 - lr: 0.0010
Epoch 4/10
1333/1333 [==============================] - 620s 465ms/step - loss: 0.4719 - accuracy: 0.7811 - val_loss: 0.4009 - val_accuracy: 0.8226 - lr: 0.0010
Epoch 5/10
1333/1333 [==============================] - 818s 614ms/step - loss: 0.4332 - accuracy: 0.8014 - val_loss: 0.4277 - val_accuracy: 0.8126 - lr: 0.0010
Epoch 6/10
1333/1333 [==============================] - 707s 531ms/step - loss: 0.4204 - accuracy: 0.8115 - val_loss: 0.4207 - val_accuracy: 0.8120 - lr: 0.0010
Epoch 7/10
1333/1333 [============

Save the model

In [10]:
model.save("model1_catsVSdogs_10epoch.h5")

C:\Users\abhay\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


 Test your model performance on custom data

In [26]:
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model

# Assuming Image_Size is defined
Image_Size = (128, 128)

results = {
    0: 'cat',
    1: 'dog'
}

# Load the saved model
model = load_model("model1_catsVSdogs_10epoch.h5")

# Load and preprocess the image
im = Image.open(r"C:\Users\abhay\OneDrive\Desktop\test1\test1\2.jpg")
im = im.resize(Image_Size)
im = np.expand_dims(im, axis=0)
im = np.array(im)
im = im / 255.0

# Predict the class of the image
pred = model.predict(im)
pred_class = np.argmax(pred, axis=1)[0]

print(pred_class, results[pred_class])

1/1 [==============================] - 1s 1s/step
1 dog
